## Plan de Notebook:

1.  **Installation**: Installer les bibliothèques nécessaires (groq, langchain, chromadb, pypdf).
2.  **Clés API**: Saisir en toute sécurité les clés API Groq et Hugging Face.
3.  **Préparation des données**:
    *   Charger les données à partir d'un fichier PDF.
    *   Diviser le document en morceaux plus petits (chunks).
    *   Créer des embeddings pour les morceaux de texte.
    *   Indexer les embeddings dans une base de données vectorielle ChromaDB.
4.  **RAG - Étape 1: Récupération (Retriever)**: Définir une fonction pour récupérer les morceaux de document pertinents en fonction d'une requête.
5.  **RAG - Étape 2: Augmentation**: Créer un modèle de prompt pour augmenter la requête avec le contexte récupéré.
6.  **RAG - Étape 3: Génération**:
    *   Initialiser un modèle linguistique (LLM) à l'aide de Groq.
    *   Générer une réponse à l'aide du prompt augmenté et du LLM.

## Instalation

In [ ]:
!pip install groq
!pip install langchain-groq
!pip install langchain-huggingface
!pip install langchain-text-splitters
!pip install langchain-community
!pip install chromadb
!pip install pypdf

# Cles API (modele d'embedding / LLM)

In [ ]:
import os
from getpass import getpass

# Prompt to enter the Groq API key securely (won't show on screen)
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

In [ ]:
# Prompt to enter the Groq API key securely (won't show on screen)
import os
from getpass import getpass
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your HF API key: ")


# Stade 0: Prepration de la base des données

## Chargement des donnees

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader("/content/Polytech-Reglement_etudes_PeiP-2023-2024.pdf")
docs = pdf_loader.load()

## Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(len(chunks))
print(chunks[0])

In [ ]:
# Extraire le text du chunks
texts= [doc.page_content for doc in chunks]
print(len(texts))
print(texts[0])

## Embedding

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings( model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
embeddings_vectors = embeddings.embed_documents(texts)


## Indexation dans la BD Vectorielle

In [ ]:
import chromadb

client = chromadb.Client()
client.delete_collection("polytech_reglement_interieur")
collection = client.get_or_create_collection("polytech_reglement_interieur")

In [ ]:
ids = ["id"+str(i)for i in range(len(chunks))]
collection.add(ids=ids,documents=texts,embeddings=embeddings_vectors)

# RAG-eptape1: Retriver

In [ ]:
def retriver(query_embedding):
    results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['documents']
    )

    res = [{"sentence_chunk": str(item)} for item in results['documents'][0]]
    res = [{"sentence_chunk": item.replace("\xa0", " ").strip()}
       for item in results['documents'][0]]
    return res

In [ ]:
query="parle moi de la politque d'absence"
embedded_query = embeddings.embed_query(query)
context = retriver(query_embedding=embedded_query)

In [ ]:
context

# RAG-etape 2:Augmentaion

In [ ]:

augmented_prompt = """Vous etes un assitant aux etudiants qui veulent se renseigner sur le reglement interieur de polytech Angers.
    Essayez de repondre aux requetes d'utilisateur , si tu ne connais pas la reponse ne l'invente pas. Garde un ton amical
\nMaintenant utiliser les elements de contexte suivant pour repondre a la requetes de l'utilisateur:
la requete:{query_a_repondre}
les contexte:{context_a_injecter}
  """


In [ ]:
from langchain import PromptTemplate
prompt=PromptTemplate.from_template(augmented_prompt)
augmented_executable_prompt = prompt.format( template=prompt,query_a_repondre=query, context_a_injecter=context)
print(augmented_executable_prompt)

# RAG-etape 3: Generation

In [ ]:
from langchain_groq import ChatGroq
llm  = ChatGroq(model = "openai/gpt-oss-20b")

In [ ]:
print(llm.invoke(augmented_executable_prompt).content)